In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from qap_lp.models import *
from qap_lp.conf import *

In [6]:
instance_name = 'esc16h'
kwargs = {}
msk_params = {**MSK_DEFAULT, **kwargs}
qap_params = {**QAP_DEFAULT, **kwargs}
# coefficients
A0, B0 = parse(f'{QAP_INSTANCE}/{instance_name}.dat')

# parse known solution
_, best_obj, arr = parse_sol(f'{QAP_SOL}/{instance_name}.sln')

param = QAPParam(A0, B0, best_obj, arr, **qap_params)

## projection methods

working on problem:
### exact penalty

$$\begin{aligned}
 &\min_X f  + \mu\cdot | \textrm{tr}(XX^\top) -  n| \\
 &= f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top)  \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
&X \ge 0 \\
\end{aligned}$$

or

$$\begin{aligned}
& \min_X  f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top) - \Lambda \bullet X \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
\end{aligned}$$


very likely to become a concave function, cannot be directly solved by conic solver.



#### gradient projection

- projection may be computed by mosek
- or some other methods

#### derivatives

$$\begin{aligned}
& \nabla_X F_\mu  = A^\top XB + AXB^\top - 2\mu X \quad (-\Lambda)\\
& \nabla_\mu F_\mu  = n - \textrm{tr}(XX^\top) \\
& \nabla_\Lambda F_\mu  = - X
\end{aligned}$$

#### projected derivative

$$\begin{aligned}
&\min_D ||\nabla F_\mu + D ||_F^2  \\
\mathbf{s.t.} & \\
&D e = D^\top e = 0 \\ 
&D_{ij} \ge 0 \quad \textsf{if: } X_{ij} = 0\\
\end{aligned}$$


In [22]:
# unpacking
A, B, n, m, e, E, ab = \
    param.A, param.B, param.n, param.m, param.e, param.E, param.ab
# more
mu = 10
nabla = QAPDerivativeL2Penalty(param, mu)
x = x0 = np.ones((n, n)) / n
nabla.obj(param.xo), nabla.obj(x)

(15.809523809523796, 170.98015873015873)

In [25]:
# unpacking solver parameters
max_iter = kwargs.get('max_iteration', 1000)
gd_method = kwargs.get('gd_method', msk_pd_on_dc)
st_method = kwargs.get('st_method', msk_st)
st_line_grids = kwargs.get('st_line_grids', 10)
logging_interval = kwargs.get('logging_interval', 50)

# unpacking params
n = param.n
xo = param.xo
best_obj = nabla.obj(xo)

# start iterations
for i in range(max_iter):

    _obj = nabla.obj(x)
    d0 = nabla.partial_f(x)

    # indices of active lower bound constraints
    lb_x, lb_y = np.where(x <= 1e-4)
    lb_x = lb_x.tolist()
    lb_y = lb_y.tolist()

    # do projection
    dp, m, D, constrs_lb = gd_method(
            param,
            d0,
            (lb_x, lb_y),
    )

    # evaluate norm of the projected gradient
    ndf = np.abs(dp).max()

    # fetch maximum stepsize
    stp = st_method(dp, x, param)
    if i % logging_interval == 0:
        logger.info(f'=====iteration: {i}====')
        logger.info(f"gradient norm: {ndf}")

    if stp <= 1e-6:
        #
        #         try:
        #             idx = constrs_lb.dual().argmax()
        #             lb_x.pop(idx)
        #             lb_y.pop(idx)
        #         except Exception as e:
        #             logging.exception(e)
        #             lb_x, lb_y = np.where(x <= 1e-5)
        #             lb_x = lb_x.tolist()
        #             lb_y = lb_y.tolist()
        #         logger.info(f"change active set @{i}")
        #         dp, m, D, constrs_lb = pd_on_dc(
        #             param, d0,
        #             (lb_x, lb_y)
        #         )
        break

    objs = [(i, nabla.obj(x + i / st_line_grids * stp * dp))
                    for i in range(1, st_line_grids + 1)]
    i_s, vs = min(objs, key=lambda x: x[-1])
    x = x + i_s / st_line_grids * stp * dp
    if i % logging_interval == 0:
        logger.info(f"steps: {i_s}, {vs}, {stp}")
        # update solution
        logger.info(f"obj: {vs, vs - _obj}, gap: {(vs - best_obj)/best_obj}")
        logger.info(f"trace deficiency: {n - x.dot(x.T).trace()}")


[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,275] =====iteration: 0====
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,276] gradient norm: 1.1155520951433573e-11
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,278] steps: 10, 155.21519219899707, 5602607020.514649
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,279] obj: (155.21519219899707, -15.764966531161662), gap: 8.817828422225727
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,279] trace deficiency: 15.308031264672607
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,684] =====iteration: 50====
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,685] gradient norm: 2.996702676280021
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,686] steps: 10, 147.75526723760038, 0.0003981762825331065
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:29,686] obj: (147.75526723760038, -0.04378549805568355), gap: 8.345965698763887
[qap.run.gradient_projection:INFO] [2020-09-27 21:53:

In [28]:
ndf

0.0

#### project point onto $D_n$

- projection may be computed by mosek
- or some other methods

In [ ]:
def proj_on_dn(param, x, **kwargs):
    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab
    
    model = mf.Model('projected_on_Dn')
    D = model.variable("d", [*A.shape], dom.inRange(0, 1))
    v = model.variable(1, dom.greaterThan(0.0))
    
    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, x))
    model.constraint(expr.sum(D, 0), dom.equalsTo(1))
    model.constraint(expr.sum(D, 1), dom.equalsTo(1))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [ ]:
def point_proj_naive():
    # point projection
    # - stuck at an corner point, vertex.
    # - once reached, it stays.
    for i in range(5):
        obj = nabla.obj(x, mu)
        print(f'====={i }====')
        alpha = 1/(i+1)
        d0 = nabla.partial_f(x, mu)
        x0 = x - d0 * alpha0
        xp = proj_on_dn(param, x0)
    #     # direction
    #     dx = xp - x
    #     st = gt_min(x/dx)
    #     print(f"stepsize: ", alpha, st)
    #     x -=  st * alpha * dx
        x = xp
        print(f"obj: ", nabla.original_obj(x), nabla.obj(x, mu), nabla.obj(x, mu) - obj)
        print(f"stopping: ", n - x.dot(x.T).trace())
        print(x.round(3).min())
        print(np.linalg.norm(d0))

In [ ]:
?constrs_lb

In [ ]:
idx = constrs_lb.dual().argmax()

In [ ]:
x.round(4)

In [ ]:
dp.round(4)

# eval

In [ ]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

In [ ]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [ ]:
check_obj_val(x)

In [ ]:
nabla.original_obj(xol), nabla.obj(xol, mu)